# Facial Keypoints Detection

## 0. Info
* Data Type : Image
* Task Type : Regression
* Competition Link : https://www.kaggle.com/c/facial-keypoints-detection
* Reference
  * https://www.kaggle.com/obione26/facial-keypoints-detection-keras-albumentations

## 1. Setting

In [ ]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
CONFIG = {
    'name' : 'Facial Keypoints Detection',
    'base_dir' : 'drive/Shared drives/Yoon/Project/Doing/Kaggle/'
}

## 2. Data

### 2.1. 준비

In [ ]:
data_dir = os.path.join(CONFIG['base_dir'], 'data', f"{CONFIG['name']}.zip")
!unzip -q "{data_dir}"

In [ ]:
!unzip -q training.zip
!unzip -q test.zip

### 2.2. EDA

In [ ]:
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data.head()

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,right_eye_outer_corner_x,right_eye_outer_corner_y,left_eyebrow_inner_end_x,left_eyebrow_inner_end_y,left_eyebrow_outer_end_x,left_eyebrow_outer_end_y,right_eyebrow_inner_end_x,right_eyebrow_inner_end_y,right_eyebrow_outer_end_x,right_eyebrow_outer_end_y,nose_tip_x,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,23.452872,37.389402,56.953263,29.033648,80.227128,32.228138,40.227609,29.002322,16.356379,29.647471,44.420571,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,24.472511,33.144443,53.987404,28.275949,78.634213,30.405923,42.728851,26.146043,16.865362,27.058860,48.206298,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,24.976421,36.603221,55.742526,27.570947,78.887368,32.651621,42.193895,28.135453,16.791158,32.087116,47.557263,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,25.307270,38.007903,56.433809,30.929864,77.910261,31.665725,41.671513,31.049990,20.458017,29.909343,51.885078,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,22.506110,38.305237,57.249571,30.672177,77.762945,31.737247,38.035436,30.935382,15.925870,30.672177,43.299534,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [ ]:
test_data.head()

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [ ]:
print(f'Train data : {len(train_data)}')
print(f'Test  data : {len(test_data)}')

Train data : 7049
Test  data : 1783


In [ ]:
# 결측값 확인
train_data.isnull().mean()

left_eye_center_x            0.001419
left_eye_center_y            0.001419
right_eye_center_x           0.001844
right_eye_center_y           0.001844
left_eye_inner_corner_x      0.677827
left_eye_inner_corner_y      0.677827
left_eye_outer_corner_x      0.678394
left_eye_outer_corner_y      0.678394
right_eye_inner_corner_x     0.678252
right_eye_inner_corner_y     0.678252
right_eye_outer_corner_x     0.678252
right_eye_outer_corner_y     0.678252
left_eyebrow_inner_end_x     0.677969
left_eyebrow_inner_end_y     0.677969
left_eyebrow_outer_end_x     0.684352
left_eyebrow_outer_end_y     0.684352
right_eyebrow_inner_end_x    0.677969
right_eyebrow_inner_end_y    0.677969
right_eyebrow_outer_end_x    0.682792
right_eyebrow_outer_end_y    0.682792
nose_tip_x                   0.000000
nose_tip_y                   0.000000
mouth_left_corner_x          0.678110
mouth_left_corner_y          0.678110
mouth_right_corner_x         0.677969
mouth_right_corner_y         0.677969
mouth_center

## 2.3. 전처리

In [ ]:
def prep_dataframe(dataframe):
    X = np.array([prep_image(i) for i in dataframe['Image']])
    if len(dataframe.columns) > 2:
        y = dataframe.drop('Image', axis=1).values
    else:
        y = None
    return X, y

def prep_image(image):
    image = image.split(' ')
    image = np.array(image, dtype=np.float32)
    image = image.reshape((96, 96, 1))
    image = image / 255.
    return image

In [ ]:
train_data = train_data.fillna(train_data.median())

In [ ]:
X_train, y_train = prep_dataframe(train_data)
X_test, _ = prep_dataframe(test_data)

In [ ]:
X_train.shape, y_train.shape, X_test.shape

((7049, 96, 96, 1), (7049, 30), (1783, 96, 96, 1))

## 3. Model

In [ ]:
def conv_block(x, filters, kernel_size, pooling=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    if pooling:
        x = tf.keras.layers.MaxPool2D()(x)
    return x

def build_model(output_dim=30):
    inputs = tf.keras.layers.Input((96, 96, 1))
    x = conv_block(inputs, 32, 5)
    x = conv_block(x, 64, 3)
    x = conv_block(x, 128, 3)
    x = conv_block(x, 256, 3)
    x = conv_block(x, 512, 3, pooling=False)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    outputs = tf.keras.layers.Dense(output_dim)(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        loss = 'mse',
        optimizer = tf.keras.optimizers.Adam(),
        metrics = ['mae']
    )

    return model

In [ ]:
model = build_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 96, 96, 32)        832       
_________________________________________________________________
batch_normalization_8 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 96, 96, 32)        25632     
_________________________________________________________________
batch_normalization_9 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 96, 96, 32)       

## 4. Train

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
mc = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True) 

model.fit(
    X_train, y_train,
    validation_split = 0.1,
    batch_size = 128,
    epochs = 100,
    callbacks = [es, mc]
)

Epoch 1/100
50/50 [==============================] - ETA: 0s - loss: 143.3499 - mae: 7.5653
Epoch 00001: val_loss improved from inf to 277.53790, saving model to model.h5
50/50 [==============================] - 6s 123ms/step - loss: 143.3499 - mae: 7.5653 - val_loss: 277.5379 - val_mae: 15.2750
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 26.9980 - mae: 4.0215
Epoch 00002: val_loss improved from 277.53790 to 90.73389, saving model to model.h5
50/50 [==============================] - 5s 101ms/step - loss: 26.9980 - mae: 4.0215 - val_loss: 90.7339 - val_mae: 8.5491
Epoch 3/100
50/50 [==============================] - ETA: 0s - loss: 22.2286 - mae: 3.6285
Epoch 00003: val_loss did not improve from 90.73389
50/50 [==============================] - 5s 97ms/step - loss: 22.2286 - mae: 3.6285 - val_loss: 108.1307 - val_mae: 9.6298
Epoch 4/100
50/50 [==============================] - ETA: 0s - loss: 18.3800 - mae: 3.2774
Epoch 00004: val_loss improved from 90.73389 to 

## 5. Test

In [ ]:
submission = pd.read_csv('SampleSubmission.csv')
id_table = pd.read_csv('IdLookupTable.csv')

In [ ]:
model.load_weights('model.h5')

In [ ]:
pred = model.predict(X_test)

In [ ]:
image_id = list(id_table['ImageId'] - 1)
feature_name = list(id_table['FeatureName'])
feature_id = [feature_name.index(i) for i in feature_name]

In [ ]:
location = [pred[i,j] for i,j in zip(image_id, feature_id)]
submission['Location'] = location

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()

,RowId,Location
0,1,67.972702
1,2,36.641808
2,3,29.384033
3,4,35.081280
4,5,61.670593


## 6. Result
* LB Score : 2.70099 (52 / 175)